<a href="https://colab.research.google.com/github/Alvorecer721/waymo-motion-prediction-challenge-2022-multipath-plus-plus/blob/boris%2Fnuscenes-configs/nuscenes_full_cycle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.1.1+cu101.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.1+cu101.html
  Using cached torch_scatter-2.1.1.tar.gz (107 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.1-cp310-cp310-linux_x86_64.whl size=489401 sha256=5d99401bd28379993524f84c504047abd5e954fad37d1c3ea529572c511219ca
  Stored in directory: /root/.cache/pip/wheels/ef/67/58/6566a3b61c6ec0f2ca0c2c324cd035ef2955601f0fb3197d5f
Successfully built torch-scatter


In [2]:
!pip install nuscenes-devkit matplotlib==3.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of nuscenes-devkit to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=70ad08a46a3e87bb0788e43b613e2d952e469fd7edc817f7799de211ad

In [3]:
!git clone https://github.com/Alvorecer721/waymo-motion-prediction-challenge-2022-multipath-plus-plus multipathpp
!cd multipathpp && git checkout boris/nuscenes-configs

import sys
sys.path.insert(0, '/content/multipathpp/code')

Cloning into 'multipathpp'...
remote: Enumerating objects: 518, done.
remote: Counting objects: 100% (369/369), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 518 (delta 260), reused 288 (delta 200), pack-reused 149
Receiving objects: 100% (518/518), 4.23 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (336/336), done.
Branch 'boris/nuscenes-configs' set up to track remote branch 'boris/nuscenes-configs' from 'origin'.
Switched to a new branch 'boris/nuscenes-configs'


In [ ]:
!python3 multipathpp/code/prerender/prerender_nuscenes.py \
    --data-version v1.0-mini \
    --data-path drive/MyDrive/multipathpp/nuscenes/v1.0-mini \
    --output-path drive/MyDrive/multipathpp/prerendered_nuscenes \
    --config multipathpp/code/configs/nuscenes_prerender.yaml

2023-05-30 20:21:56.114185: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:21:57.469800: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 5.401 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.
 50% 5/10 [00:15<00:12,  2.44s/it]

In [ ]:
import os


PRERENDERED_NUSCENES_PATH = 'drive/MyDrive/multipathpp/prerendered_nuscenes'
train_files = [filename for filename in os.listdir(PRERENDERED_NUSCENES_PATH) if int(filename.split('_')[1]) < 8]
val_files = [filename for filename in os.listdir(PRERENDERED_NUSCENES_PATH) if int(filename.split('_')[1]) >= 8]

os.makedirs(os.path.join(PRERENDERED_NUSCENES_PATH, 'train'))
os.makedirs(os.path.join(PRERENDERED_NUSCENES_PATH, 'val'))

for f in train_files:
  os.rename(os.path.join(PRERENDERED_NUSCENES_PATH, f), os.path.join(PRERENDERED_NUSCENES_PATH, 'train', f))

for f in val_files:
  os.rename(os.path.join(PRERENDERED_NUSCENES_PATH, f), os.path.join(PRERENDERED_NUSCENES_PATH, 'val', f))

!ls drive/MyDrive/multipathpp/prerendered_nuscenes

train  val


In [14]:
!python3 multipathpp/code/normalization.py \
    --data-path drive/MyDrive/multipathpp/prerendered_nuscenes/train \
    --output-path drive/MyDrive/multipathpp/normalization/normalization_coefs_nuscenes.npy \
    --config multipathpp/code/configs/normalization.yaml

100% 1/1 [00:11<00:00, 11.65s/it]


In [46]:
!python3 multipathpp/code/train.py \
    --train-data-folder drive/MyDrive/multipathpp/prerendered_nuscenes/train \
    --val-data-folder drive/MyDrive/multipathpp/prerendered_nuscenes/val \
    --norm-coeffs drive/MyDrive/multipathpp/normalization/normalization_coefs_nuscenes.npy \
    --epoch 1

2023-05-30 15:27:03.545255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 15:27:05.823623: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/multipathpp
